In [4]:
import cellpylib
import pandas
import numpy
import os

In [5]:
# Generator/Selector of PIs
def instances(PI_num, lib, PI_objs_num):
    os.chdir(r"C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Experiments\Balanced KP Instances 2024\Training")
    # When knapsack limit undefined, an instance from the library is used
    # fileName = f"GA-{lib}_20_0{PI_num}.kp"
    # fileName = f"GA-{lib}_50_0{PI_num}.kp" # Introduces the number of
    # objects into the filename that will be requested from the library
    fileName = f"GA-{lib}_EASY_{PI_objs_num}_{PI_num}.kp"
    f = open(fileName, "r") # Opening, reading, and cleaning the instance
    lines = f.readlines()
    line = lines[0].split(",")
    nbItems = int(line[0].strip())
    k_limit = int(line[1].strip())
    PI = [None] * nbItems
    for i in range(0, nbItems):
        line = lines[i + 1].split(",")
        weight = int(line[0].strip())
        profit = float(line[1].strip())
        PI[i] = (profit, weight) # Saves objects as (profit, weight)
    return PI, k_limit # Returns the instance and the knapsack limit 

In [6]:
def CA(rule, seed = None):
    if seed == None:
        init = cellpylib.init_random(100)
    else:
        init = numpy.array([seed])
    cellular_automaton = cellpylib.evolve(init,
                                          timesteps = 100,
                                          memoize = True,
                                          apply_rule = lambda 
                                          n,
                                          c, 
                                          t: cellpylib.nks_rule(n, rule))
    return cellular_automaton

In [7]:
# Maximum Profit per Weight Unit Solution Generator
def MAPW(PI, k_limit):
    number_objects = len(PI)
    wgt = 0
    MAPW = []
    solution = [0] * number_objects # Creates solution template of 0s
    for i in range(number_objects):
        MAPW.append((i, PI[i][0], PI[i][1], PI[i][0]/PI[i][1])) # Adds a new 
        # indexed problems list
    MAPW = sorted(MAPW, reverse = True, key = lambda x: x[3]) # Sorts the new list
    # by weight
    for object in MAPW:
        if object[2] > k_limit:
            continue
        wgt += object[2] # Evaluates the acumulated weight
        if wgt <= k_limit: # When the knapsack is broken, omits the object
            solution[object[0]] = 1 # Adds the most profitable object until full
        else:
            wgt -= object[2]
    return solution

In [8]:
# Minimum Weight Heuristic Solution Generator
def MIW(PI, k_limit):
    number_objects = len(PI)
    wgt = 0
    MIW = []
    solution = [0] * number_objects # Creates solution template of 0s
    for i in range(number_objects):
        MIW.append((i, PI[i][0], PI[i][1])) # Adds a new indexed problems list
    MIW = sorted(MIW, key = lambda x: x[2]) # Sorts the new list by weight
    for object in MIW:
        if object[2] > k_limit:
            continue
        wgt += object[2] # Evaluates the acumulated weight
        if wgt <= k_limit: # When the knapsack is broken, omits the object
            solution[object[0]] = 1 # Adds the lighter object until full
        else:
            wgt -= object[2]
    return solution

In [9]:
# Evaluator of Solutions
def evaluator(PI, solution, k_limit):
    s1 = (0, 0, 0, 0)
    for i in range(len(PI)): # Iterates all the objects in the problem instance
        if solution[i] == 1: # When the object is in the knapsack considers the 
            # object in the evaluation
            s1 = (k_limit, s1[1] + PI[i][0], s1[2] + PI[i][1]) # Sums up the profit and 
            # the weight of all the items
    if s1[2] <= k_limit: # When the knapsack is not broken saves a record of 0
        s1 = (k_limit, s1[1], s1[2], 0, solution)
    else:
        s1 = (k_limit, s1[1], s1[2], 1, solution) # When the knapsack is broken saves a record of 1
    return s1

In [10]:
# Cellular Automata Solution Generator
def SolutionAssembler(PI, k_limit, strategy):
    wgt = 0
    solution = [0] * len(PI)
    for i in range(len(strategy)):
        solvers = [MIW(PI[i:], k_limit), MAPW(PI[i:], k_limit)]
        result = solvers[strategy[i]][0]
        if PI[i][1] > k_limit:
            continue
        if result == 1:
            wgt += PI[i][1]
        if wgt <= k_limit:
            solution[i] = result
        else:
            wgt -= PI[i][1]
    return solution

In [11]:
# Cellular Automata Solution Generator
def SolutionAssembler(PI, k_limit, strategy):
    solvers = [MAPW(PI, k_limit), MIW(PI, k_limit)]
    solution = [0] * len(PI)
    wgt = 0
    for i in range(len(strategy)):
        result = solvers[strategy[i]]
        if PI[i][1] > k_limit:
            continue
        if result[i] == 1:
            wgt += PI[i][1]
        if wgt <= k_limit:
            solution[i] = result[i]
        else:
            wgt -= PI[i][1]
    return solution

In [12]:
def seeds_generator(heuristic_index):
    # instance_libraries = [('DEF', '100'), ('MARK', '25'), ('MAXP', '100'), ('MAXPW', '25'), ('MINW', '100')]
    instance_libraries = [('DEF', '100'), ('MAXP', '100'), ('MAXPW', '100'), ('MINW', '100')]
    PI_nums = [str(num).zfill(3) for num in range(25)]
    evaluations = []
    PI_label_nums = []
    for lib, num in instance_libraries:
        for i in range(len(PI_nums)):
            PI_label_nums.append(f'{lib}_50_{str(i + 1).zfill(2)}')
            PI, k_limit = instances(PI_nums[i], lib, num)
            solvers = [MIW(PI, k_limit), MAPW(PI, k_limit)]
            solution = solvers[heuristic_index]
            evaluations.append(evaluator(PI, solution, k_limit))
        df = pandas.DataFrame(evaluations, columns = ['Knapsack Limit', 'Profit', 'Weight', 'Knapsack State', 'Solution'])
        df.index = PI_label_nums
        df.index.names = ['Problem Instance']
    return df

In [13]:
def solver(heuristics):
    # instance_libraries = [('DEF', '100'), ('MARK', '25'), ('MAXP', '100'), ('MAXPW', '25'), ('MINW', '100')]
    instance_libraries = [('DEF', '100'), ('MAXP', '100'), ('MAXPW', '100'), ('MINW', '100')]
    PI_nums = [str(num).zfill(3) for num in range(25)]
    results = {}
    PI_label_nums = []
    for rule in range(1, 256):
        evaluations = []
        i = 0
        for lib, num in instance_libraries:
            for j in range(len(PI_nums)):
                PI, k_limit = instances(PI_nums[j], lib, num)                
                solution = SolutionAssembler(PI, k_limit, heuristics[str(rule)][i])
                evaluations.append(evaluator(PI, solution, k_limit)[1])                    
                i += 1
        results[rule] = evaluations            
    return results

In [14]:
seeds = seeds_generator(1).Solution
heuristics = {}
for rule in range(1, 256):
    strategies = []
    for seed in seeds:
        strategies.append(CA(rule, seed)[-1])
    heuristics[str(rule)] = strategies

In [20]:
results = solver(heuristics)
sum_results = {key: sum(value) for key, value in results.items()}

In [21]:
best_heuristic = max(sum_results, key = sum_results.get)
best_performance_score = sum_results[best_heuristic]
print(f'Best CA heuristic: {best_heuristic}, Performance: {best_performance_score}')

Best CA heuristic: 8, Performance: 89181.0


In [22]:
MIW_results = seeds_generator(0).Profit
MAXPW_results = seeds_generator(1).Profit
print('MIW Performance:' + str(sum(MIW_results)), 'MAXPW Performance:' + str(sum(MAXPW_results)))

MIW Performance:67522.0 MAXPW Performance:89181.0


In [ ]:
# def HHRun():
#     heuristics = {}
#     for rule in range(252, 253):
#         strategies = []
#         for _ in range(100):
#             strategies.append(CA(rule)[-1])
#         heuristics[str(rule)] = strategies
#     results = solver(heuristics)
#     sum_results = {key: sum(value) for key, value in results.items()}
#     best_heuristic = max(sum_results, key = sum_results.get)
#     best_performance_score = sum_results[best_heuristic]
#     return (sum_results, best_heuristic, best_performance_score)

In [ ]:
# def PBMH():
#     CA = [HHRun() for _ in range(100)]
#     acum = []
#     for _ in range(100):
#         # Create a copy of the current cellular automata generation
#         new_CA = CA.copy()
#         for i in range(len(CA)):
#             # Create new solution for this cell
#             new_solution = HHRun()
#             if new_solution[2] > CA[i][2]:
#                 new_CA[i] = new_solution
#         # Update cellular automata with new generation
#         CA = new_CA
#         acum.append(CA)
#     # Calculate the profits for the last generation
#     profits = [solution[2] for solution in CA]

#     # Select the best solution (highest profit) from the last generation
#     best_solution = CA[profits.index(max(profits))]
#     # return acum
#     return best_solution

In [ ]:
# BS = PBMH()

In [ ]:
# print(BS[1], BS[2])

In [23]:
# heuristics = {}
# for rule in range(1, 256):
#     strategies = []
#     for _ in range(100):
#         strategies.append(CA(rule)[-1])
#     heuristics[str(rule)] = strategies

In [24]:
# results = solver(heuristics)
# sum_results = {key: sum(value) for key, value in results.items()}

In [25]:
# best_heuristic = max(sum_results, key = sum_results.get)
# best_performance_score = sum_results[best_heuristic]
# print(f'Best CA heuristic: {best_heuristic}, Performance: {best_performance_score}')

Best CA heuristic: 8, Performance: 89181.0


In [26]:
# MIW_results = seeds_generator(0).Profit
# MAXPW_results = seeds_generator(1).Profit
# print('MIW Performance:' + str(sum(MIW_results)), 'MAXPW Performance:' + str(sum(MAXPW_results)))

MIW Performance:67522.0 MAXPW Performance:89181.0
